In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/audio-features-and-lyrics-of-spotify-songs/spotify_songs.csv')
df=df.drop(columns = ['track_album_id','playlist_id']) #removing unique identifiers
df.head(10)

In [ ]:
df.describe().round(decimals=2)

In [ ]:
df.shape, df.columns, df.info()

In [ ]:
type(df['language'].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig,ax = plt.subplots(figsize=(20,10))
ax = sns.countplot(x='language', data=df)

In [ ]:
dfen = df[df['language'] == 'en']
dfen['language'].value_counts()

In [ ]:
dfen.hist(figsize = (15,15))
plt.show()

In [ ]:
dfen.isnull().sum()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(dfen.corr(), annot=True)

In [ ]:
dfen['track_artist'].value_counts().head(10).plot(kind='barh')

In [ ]:
dfen['playlist_subgenre'].value_counts().head(10).plot(kind='barh')

In [ ]:
dfen = dfen.sort_values('track_popularity', ascending = False)
dfen

In [ ]:
plt.figure(figsize=(20,10))
sns.set(style='whitegrid')
x = dfen.groupby('track_artist')['track_popularity'].sum().sort_values(ascending=False).head(10)
ax = sns.barplot(x,x.index)

In [ ]:
dfen

In [ ]:
dfen.columns

In [ ]:
dfgroup = dfen.groupby(['track_artist'], as_index = False).sum().sort_values('track_popularity', ascending=False)
dfgroup.head(10)

In [ ]:
top10artist = dfgroup['track_artist'].head(10).tolist()
top10artist

In [ ]:
dfentop10 = dfen[dfen['track_artist'].isin(top10artist)]
dfentop10

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(dfentop10.corr(), annot=True)

In [ ]:
dfentop10.describe()

In [ ]:
dfentop10.hist(figsize = (20,20))
plt.show()

In [ ]:
#multiple ANOVA
#wordcloud
#correlation
#compare means
#group queen and not queen

In [ ]:
parameter = dfentop10.select_dtypes(include=['int64', 'float']).columns
parameter

In [ ]:
for i in parameter:
    plt.figure(figsize=(20,10))
    sns.boxplot(x='track_artist', y=i, data=dfentop10)

In [ ]:
#creating new column 'grou' to differentiate 'Queen' and others

dfentop10['group'] = ['Queen' if x == 'Queen' else 'Not Queen' for x in dfentop10['track_artist']]
dfentop10

In [ ]:
!pip install pingouin

In [ ]:
import pingouin as pg
# Compute the two-way mixed-design ANOVA
for i in parameter:
    print(i)
    aov = pg.anova(dv=i, between='group', data=dfentop10)
    pg.print_table(aov)

In [ ]:
for i in parameter:
    plt.figure(figsize=(10,10))
    sns.boxplot(x='group', y=i, data=dfentop10)

In [ ]:
queen = dfentop10.loc[dfentop10['group'] == 'Queen']['lyrics']
nqueen = dfentop10.loc[dfentop10['group'] == 'Not Queen']['lyrics']
queen = queen.values.tolist()
nqueen = nqueen.values.tolist()

from gensim.parsing.preprocessing import remove_stopwords

queenstr = remove_stopwords(str(queen))
nqueenstr = remove_stopwords(str(nqueen))

In [ ]:
from wordcloud import WordCloud
plt.figure(figsize=(20,20))
wordcloud = WordCloud(width = 3000, height = 2000).generate(queenstr)
plt.imshow(wordcloud)

In [ ]:
plt.figure(figsize=(20,20))
wordcloud = WordCloud(width = 3000, height = 2000).generate(nqueenstr)
plt.imshow(wordcloud)

In [ ]:
df.columns

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#Creating Random Forest
y = df['track_popularity']
columns = ['danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
x = df[columns]

train_x, val_x, train_y, val_y = train_test_split(x,y, random_state=1)

popularity_model = RandomForestRegressor(max_leaf_nodes=50, random_state=1)
popularity_model.fit(train_x, train_y)

popularity_prediction = popularity_model.predict(val_x)
popularity_mae = mean_absolute_error(popularity_prediction, val_y)

#test
print(popularity_model.predict(df[columns].head()))
print('==============================================')
print(df['track_popularity'].head())
print('Popularity MAE:', popularity_mae)